<a href="https://colab.research.google.com/github/felippedamasoade/CORRE-O-DE-CATEGORIAS-DE-DESPESAS-EM-PLANILHAS/blob/main/Random_Forest_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Bibliotecas
import pandas as pd
import numpy as np
import random
import spacy
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression

In [ ]:
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 13.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# Criação da base de dados - Definindo categorias e descrições associadas
categorias_descrições = {
    'Estoque': ['Compra de materiais', 'Aquisição de novos produtos', 'Estoque de segurança'],
    'Marketing': ['Campanha no Google Ads', 'Produção de material publicitário', 'Anúncios em redes sociais'],
    'Utilidades': ['Pagamento da conta de luz', 'Conta de água do escritório', 'Telefone e internet'],
    'Manutenção': ['Manutenção de equipamentos de ar-condicionado', 'Reparo de mobiliário', 'Serviço de limpeza'],
    'Salários': ['Folha de pagamento mensal', 'Adiantamento quinzenal', 'Bônus de final de ano'],
    'Tecnologia': ['Renovação de licença de software', 'Upgrade de equipamentos de TI', 'Suporte técnico'],
    'Serviços Profissionais': ['Honorários de consultoria', 'Serviços legais', 'Contabilidade externa']
}

# Gerar 500 registros
dados = []
for _ in range(500):
    categoria = random.choice(list(categorias_descrições.keys()))
    descrição = random.choice(categorias_descrições[categoria])
    valor = round(random.uniform(100, 5000), 2)
    dados.append([descrição, categoria, valor])

# Criar DataFrame
df_pagamentos = pd.DataFrame(dados, columns=['Descrição', 'Categoria', 'Valor'])

# Visualizando as primeiras linhas dos dados
print(df_pagamentos.head())

# Salvando o DataFrame em um arquivo CSV
df_pagamentos.to_csv('dados_pagamentos.csv', index=False)

                           Descrição               Categoria    Valor
0             Adiantamento quinzenal                Salários  2337.90
1              Contabilidade externa  Serviços Profissionais  2505.42
2              Contabilidade externa  Serviços Profissionais  1233.44
3  Produção de material publicitário               Marketing  3063.75
4          Folha de pagamento mensal                Salários  2810.39


In [ ]:
#Data Frame
df_pagamentos

,Descrição,Categoria,Valor
0,Adiantamento quinzenal,Salários,2337.90
1,Contabilidade externa,Serviços Profissionais,2505.42
2,Contabilidade externa,Serviços Profissionais,1233.44
3,Produção de material publicitário,Marketing,3063.75
4,Folha de pagamento mensal,Salários,2810.39
...,...,...,...
495,Estoque de segurança,Estoque,4891.10
496,Adiantamento quinzenal,Salários,258.63
497,Manutenção de equipamentos de ar-condicionado,Manutenção,3674.25
498,Manutenção de equipamentos de ar-condicionado,Manutenção,631.83


In [ ]:
# Olhando os 5 primeiros dados
df_pagamentos.head()

In [ ]:
df_pagamentos.tail

<bound method NDFrame.tail of                                          Descrição               Categoria  \
0                           Adiantamento quinzenal                Salários   
1                            Contabilidade externa  Serviços Profissionais   
2                            Contabilidade externa  Serviços Profissionais   
3                Produção de material publicitário               Marketing   
4                        Folha de pagamento mensal                Salários   
..                                             ...                     ...   
495                           Estoque de segurança                 Estoque   
496                         Adiantamento quinzenal                Salários   
497  Manutenção de equipamentos de ar-condicionado              Manutenção   
498  Manutenção de equipamentos de ar-condicionado              Manutenção   
499               Renovação de licença de software              Tecnologia   

       Valor  
0    2337.90  
1    2505.42  
2    1233.44  
3    3063.75  
4    2810.39  
..       ...  
495  4891.10  
496   258.63  
497  3674.25  
498   631.83  
499  2800.69  

[500 rows x 3 columns]>

In [ ]:
# carregar o modelo Glove do Spacy
nlp = spacy.load('en_core_web_md')

In [ ]:
nlp

In [ ]:
# função para converter descrições em vetores
def get_vectors(text):
    return nlp(text).vector

In [ ]:
# A função get_vectors recebe um texto, aplica o modelo nlp que converte o
# texto em um documento processado pelo spaCy e extrai o vetor médio que representa todo o documento.
# aplicar a função de vetorização nas descriçõe
# df['Descrição'].apply(get_vectors) aplica essa função a cada descrição no DataFrame, resultando em uma nova
# coluna Vectors que contém os vetores correspondentes às descrições.

df_pagamentos['Vectors'] = df_pagamentos['Descrição'].apply(get_vectors)

In [ ]:
# preparar dados para o modelo
X = np.array(list(df_pagamentos['Vectors']))
y = df_pagamentos['Categoria']

In [ ]:
# X contém os vetores (features) convertidos a partir das descrições.
# y é o vetor de saída que contém as categorias das despesas.
X

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.65105   , -0.6704    , -1.29675   , ..., -2.0462    ,
        -0.9219    ,  1.254     ],
       [-0.65105   , -0.6704    , -1.29675   , ..., -2.0462    ,
        -0.9219    ,  1.254     ],
       ...,
       [ 2.2833714 , -5.062886  , -0.30234712, ...,  1.9158287 ,
         0.8271001 , -5.193471  ],
       [ 2.2833714 , -5.062886  , -0.30234712, ...,  1.9158287 ,
         0.8271001 , -5.193471  ],
       [ 3.091656  , -7.831398  ,  0.947336  , ...,  3.2690403 ,
         1.8549601 , -4.23696   ]], dtype=float32)

In [ ]:
# Dividir os dados em treinamento e test
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state=42) # 20% para teste e ,  garante que os splits sejam reproduzíveis.

In [ ]:
# treinar um modelo de classificação
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train,y_train)

RandomForestClassifier()

In [ ]:
# Avaliar o modelo
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

                        precision    recall  f1-score   support

               Estoque       1.00      1.00      1.00        13
            Manutenção       0.76      1.00      0.86        19
             Marketing       1.00      1.00      1.00        10
              Salários       1.00      1.00      1.00        12
Serviços Profissionais       1.00      0.60      0.75        15
            Tecnologia       1.00      1.00      1.00        14
            Utilidades       1.00      1.00      1.00        17

              accuracy                           0.94       100
             macro avg       0.97      0.94      0.94       100
          weighted avg       0.95      0.94      0.94       100



In [ ]:
# calculando a acuracia
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia do Modelo: {accuracy * 100:.2f}%")

Acurácia do Modelo: 94.00%


MAtriz de Confusão


In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, f1_score, roc_auc_score

# Supondo que X e y sejam seus dados e rótulos
# Divisão dos dados
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinamento do modelo
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

# Previsões
y_pred_log_reg = log_reg.predict(X_test)

# Matriz de Confusão
cm_log_reg = confusion_matrix(y_test, y_pred_log_reg)
print("Matriz de Confusão - Regressão Logística:")
print(cm_log_reg)

# Métricas
accuracy_log_reg = accuracy_score(y_test, y_pred_log_reg)
recall_log_reg = recall_score(y_test, y_pred_log_reg, average='weighted')
f1_log_reg = f1_score(y_test, y_pred_log_reg, average='weighted')

# AUC-ROC para múltiplas classes
auc_log_reg = roc_auc_score(y_test, log_reg.predict_proba(X_test), multi_class='ovr')

print(f"Precisão (Accuracy): {accuracy_log_reg}")
print(f"Recall: {recall_log_reg}")
print(f"F1-score: {f1_log_reg}")
print(f"AUC-ROC: {auc_log_reg}")


Matriz de Confusão - Regressão Logística:
[[13  0  0  0  0  0  0]
 [ 0 19  0  0  0  0  0]
 [ 0  0 10  0  0  0  0]
 [ 0  0  0 12  0  0  0]
 [ 0  6  0  0  9  0  0]
 [ 0  0  0  0  0 14  0]
 [ 0  0  0  0  0  0 17]]
Precisão (Accuracy): 0.94
Recall: 0.94
F1-score: 0.936590909090909
AUC-ROC: 0.994468524251806


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
